### Adjust Word Level

In [ ]:
#!pip install xlsxwriter

In [1]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from pathlib import Path
import shutil

In [2]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


#### While Loop

In [3]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
lang_pair = "French"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# prefix suffix file
prefix_suffix = True  # True, False  # True for adding prefix suffix word

In [4]:
Path(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Deployment/Result/3-Adjust Word Level/{lang_folder.capitalize()} {lang_pair.capitalize()}").mkdir(parents=True, exist_ok=True)  # create path

##### Dependency DataFrames

In [5]:
df_word_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_word_all

,word,frequency
0,bir,18835735
1,bu,11062659
2,ne,8025880
3,ve,7766036
4,için,5484109
...,...,...
988212,karneleme,5
988213,karnaya,5
988214,dörtlümüzün,5
988215,karnavalınız,5


In [6]:
#df_twogram_sent = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/Merge/Twogram_Merge.csv")
df_twogram_sent = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/N Gram And Sentence/Two_Gram_And_Sentence.csv")  # ext. sentence and ngram
df_twogram_sent.rename(columns={"two_gram":"twogram"}, inplace=True)  # ext. Not: Two_Gram_And_Sentence.csv convert to Two_Gram_And_Sentence_All.csv
df_twogram_sent

,twogram,frequency
0,teşekkür ederim,244149
1,öyle mi,209900
2,ne oldu,195799
3,aman tanrım,189521
4,özür dilerim,153784
...,...,...
1036515,güzeldi tommy,3
1036516,durumu tuhaflaştırma,3
1036517,güzeldi canım,3
1036518,güzeldi daniel,3


In [7]:
#lang_pair_list = glob.glob(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Result/{lang_folder.capitalize()}/{lang_folder.capitalize()}_And_{lang_pair.lower().capitalize()}*_All.xlsx")
#lang_pair_list

In [8]:
#df_pair_ety = pd.read_excel(f"{lang_pair_list[0]}")  # need only dict_entry_main column
df_pair_ety = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Result/{lang_folder.capitalize()}/{lang_folder.capitalize()} {lang_pair.lower().capitalize()}/{lang_folder.capitalize()}_{lang_pair.lower().capitalize()}_Shared_Vocabulary.xlsx")
df_pair_ety

,dict_entry_main,french_word
0,abajur,abat-jour
1,abaküs,abacus
2,abandone,abandonner
3,abiye,habillé
4,abluka,blocus
...,...,...
1593,yeniçeri,janissaire
1594,yoğurt,yaourt
1595,zebra,zèbre
1596,zombi,zombie


In [9]:
# Option
if prefix_suffix:
    df_prefix_suffix_select = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Result/{lang_folder.capitalize()}/{lang_folder.capitalize()} {lang_pair.lower().capitalize()}/{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Word_Prefix_Suffix_Custom_Result.xlsx")
    df_prefix_suffix_select = df_prefix_suffix_select.loc[:,["search_word","word"]]
    df_prefix_suffix_select.rename(columns={"search_word":"dict_entry_main"}, inplace=True)
    df_pair_merge = pd.merge(df_pair_ety,df_prefix_suffix_select, how="inner", on="dict_entry_main")
    df_pair_merge.drop_duplicates(inplace=True)
    df_pair_merge.reset_index(drop=True, inplace=True)
    df_pair_merge = df_pair_merge.loc[:,["word",f"{lang_pair.lower()}_word"]]
    df_pair_merge.rename(columns={"word":"dict_entry_main"}, inplace=True)
    df_pair_all = df_pair_merge
else:
    df_pair_all = df_pair_ety

df_pair_all

,dict_entry_main,french_word
0,abajur,abat-jour
1,abaküs,abacus
2,abandoned,abandonner
3,abiye,habillé
4,abluka,blocus
...,...,...
5385,zombileri,zombie
5386,zombiye,zombie
5387,zürafa,girafe
5388,zürafalar,girafe


In [10]:
disable_list = ["sex","seks","seksi","sexy","sexe","seksüel","sexuell","gey","gay","lezbiyen","lesbienne","eşcinsel","mastürbasyon","masturbation","erotik","érotique", \
"bikini","penis","vagina","vajina","fetish","fetiş","fetishy","erotic","erotik","sexdom","kondom","condom","dildo","fetisj","hétérosexuel","féticher","fétiche","homosexuel"\
"ereksiyon","erectie","erection","érection","homoseksüel","prezervatif","préservatif","ass","fetisch","fetiche"]

In [11]:
words = df_pair_all["dict_entry_main"].values.tolist()
disable_set = set(disable_list)
words_set = set(words)

In [12]:
df_pair = pd.DataFrame(list(words_set.difference(disable_set)), columns=["dict_entry_main"])
df_pair

,dict_entry_main
0,makyajımı
1,galanın
2,reyonda
3,garland
4,kanepe
...,...
5248,demokrat
5249,veteriner
5250,zombiler
5251,rekoru


##### Dependency Functions

In [13]:
def remove_repetition(word_group):
    '''
    remove_repetition(word_group): detect word repetion in word group 
    '''
    words = word_tokenize(word_group)
    word_unique = set(words)
    if len(word_unique) == 1:
        return "repetitive_word_group"
    else:
        return word_group

In [14]:
def word_count_result(df,column_list): # df is dataframe, column_list is list value
    '''
    word_count_bool(df, column_list): df columns word count for word frequency\n
    df is dataframe, column_list is list value\n
    word_count_bool(df, ["word","twogram"]):
    '''
    list_all = []
    for i in df.loc[:,[x for x in column_list]].columns:
        var_list = df[f"{i}"].dropna().tolist()
        for j in var_list:
            list_all.append(j)
    text = " ".join(list_all)
    word_list = re.findall(r"\w+",text, re.UNICODE)
    df_word_list = pd.DataFrame(word_list, columns=["word"])
    #df_word_list.rename(columns={0:"word"}, inplace=True)
    df_word_count = pd.DataFrame(df_word_list.value_counts())
    df_word_count.reset_index(inplace=True)
    df_word_count.rename(columns={0:"word_count"}, inplace=True)
    
    return  df_word_count

In [15]:
def word_count_bool(df, word_thresh_num, column_list): # df is a dataframe, word_thresh_num is an integer, column_list is list value
    '''
    word_count_bool(df, word_thresh_num, column_list):\n
    df is a dataframe, word_thresh_num is an integer, column_list is df column names\n
    word_count_bool(df, 7, ["word","twogram"]):
    '''
    list_all = []
    for i in df.loc[:,[x for x in column_list]].columns:
        var_list = df[f"{i}"].dropna().tolist()
        for j in var_list:
            list_all.append(j)
    text = " ".join(list_all)
    word_list = re.findall(r"\w+",text, re.UNICODE)
    df_word_list = pd.DataFrame(word_list, columns=["word"])
    #df_word_list.rename(columns={0:"word"}, inplace=True)
    df_word_count = pd.DataFrame(df_word_list.value_counts())
    df_word_count.reset_index(inplace=True)
    df_word_count.rename(columns={0:"count"}, inplace=True)
    
    return  df_word_count["count"][df_word_count.loc[:,"count"] > word_thresh_num].any()  

#### Condition 1

In [16]:
def condition1(word_start_num = 0, word_limit_num = 28, thresh_num = 2, total_part_num = 4):
    '''
    default parameter:\n    
    condition1(word_start_num = 0, word_limit_num = 28, thresh_num = 2, total_part_num = 4) \n
    word_end 28/4 = 7 word group
    '''

    # while loop code block word and twogram pair
    word_thresh_num = thresh_num  # want how many word sample 
    word_start = word_start_num  # 0
    word_end = int((word_limit_num-word_start_num)/total_part_num)  
    step_num = word_end  
    word_limit = word_limit_num   
    part_num = 1  # first output file extention
    
    #twogram_num = word_thresh_num * step_num  # word_thresh_num*step_num minimum: for each word takes two twogram
    twogram_pair_num = word_thresh_num * step_num  # word_thresh_num*step_num minimum: for each word takes two twogram pair
    
    while word_end <= word_limit:
        df_word = df_word_all.iloc[word_start:word_end,]  # must be include word and frequency column
        df_word.reset_index(drop=True, inplace=True)
    
        # language pair twogram
        word_pair_list = df_pair["dict_entry_main"].to_list()  # *****
        word_list = df_word["word"].to_list()  # *****    
        ngram_list = []
        for i in word_pair_list:
            for j in word_tokenize(i):
                for k in word_list:
                    twogram_1_2 = f"{j} {k}"
                    ngram_list.append(twogram_1_2)
                    twogram_2_1 = f"{k} {j}"
                    ngram_list.append(twogram_2_1)
        df_pair_ngram = pd.DataFrame(ngram_list, columns=["twogram"])
        #df_pair_ngram.rename(columns={0:"twogram"}, inplace=True)  # ******
        df_pair_ngram.iloc[:,0] = df_pair_ngram.iloc[:,0].apply(lambda x: remove_repetition(x))
        df_pair_ngram.drop_duplicates(inplace=True)
        df_pair_ngram.reset_index(drop=True, inplace=True)
        df_lang_pair_twogram = pd.merge(df_twogram_sent, df_pair_ngram, how="inner", on="twogram")
        df_lang_pair_twogram.rename(columns={"twogram":f"twogram_pair_{lang_pair.lower()}"}, inplace=True)
        df_lang_pair_twogram.drop_duplicates(inplace=True)
        #df_lang_pair_twogram = df_lang_pair_twogram.head(100)
    
        # output
        df_output_result = pd.concat([df_word, df_lang_pair_twogram], axis=1)
    
        df_lesson_result = pd.DataFrame(columns=["word","freq_word",f"twogram_pair_{lang_pair.lower()}",f"freq_twogram_pair_{lang_pair.lower()}"])
        a = 0
        #for i in range(0,110):
        for i in range(len(df_output_result)):  # *****
            # Insert words and their count 
            try:
                word = df_output_result.iloc[i,0]  # word 
                freq_word = df_output_result.iloc[i,1]  # word freq
                df_lesson_result.loc[i,"word"] = word
                df_lesson_result.loc[i,"freq_word"] = freq_word
            except:
                pass
            
            # Insert twogram pair
            try:
                var2 = df_output_result.loc[a,f"twogram_pair_{lang_pair.lower()}"]
                freq_var2 = df_output_result.iloc[a,3]  # twogram_pair frequency
                if (len(df_lesson_result[f"twogram_pair_{lang_pair.lower()}"]) < twogram_pair_num): # and (not(word_count(df_lesson_result, word_thresh_num))):
                    df_lesson_result.loc[i,f"twogram_pair_{lang_pair.lower()}"] = var2
                    df_lesson_result.loc[i,f"freq_twogram_pair_{lang_pair.lower()}"] = freq_var2
                    try:
                        while word_count_bool(df_lesson_result, word_thresh_num, [f"twogram_pair_{lang_pair.lower()}"]): # word count result                
                            a += 1
                            var2 = df_output_result.loc[a,f"twogram_pair_{lang_pair.lower()}"]
                            freq_var2 = df_output_result.iloc[a,3]  # twogram_pair frequency
                            df_lesson_result.loc[i,f"twogram_pair_{lang_pair.lower()}"] = var2
                            df_lesson_result.loc[i,f"freq_twogram_pair_{lang_pair.lower()}"] = freq_var2
                        else:
                            pass
                    except:
                        df_lesson_result.loc[i,f"twogram_pair_{lang_pair.lower()}"] = np.nan
                        df_lesson_result.loc[i,f"freq_twogram_pair_{lang_pair.lower()}"] = np.nan
                else:
                    pass
            except:
                pass
            a += 1
    
        df_lesson_word_count = word_count_result(df_lesson_result, [f"twogram_pair_{lang_pair.lower()}"])
        df_lesson_result = pd.merge(df_lesson_result, df_lesson_word_count, how="left", on="word")
        df_lesson_result = df_lesson_result.drop_duplicates()
        df_lesson_result.to_excel(f"{lang_folder}_{lang_pair}_{word_limit}_Word_Step_{step_num}_Result{part_num}.xlsx", index=False)
    
        word_start += step_num
        word_end += step_num
        part_num += 1 

#### Condition 2

In [17]:
# --- Parameter Part ---

# condition 1
condition1_word_start = 0
condition1_word_limit = 28
condition1_word_thresh_num = 2
condition1_total_part_num = 4  # 28/4 = 7 word group

# condition 2
condition1_dependency = True  # True, False
word_thresh_num = 10
twogram_thresh_minus = 0  # for optional twogram thresh number 
twogram_pair_thresh_minus = 0  # for optinal twogram pair thresh number.

word_start = 0  # 0
word_end = 28  # 10
step_num = word_end  # 10
word_limit = word_end  # 200
part_num = 1
    
if condition1_dependency:        
    # Read previous part result
    condition1(word_start_num = condition1_word_start, word_limit_num = condition1_word_limit, thresh_num = condition1_word_thresh_num, total_part_num = condition1_total_part_num)  # word_end 28/4 = 7 word group. Condition 1 parameters 
    df_part_all = pd.DataFrame()
    part_result_file = glob.glob(f"{lang_folder}_{lang_pair}_*_Word_Step_*_Result*.xlsx")
    for i in part_result_file:
        df_var = pd.read_excel(f"{i}")
        df_part_all = pd.concat([df_part_all,df_var], axis=0)
    df_part_twogram_pair = df_part_all.loc[:,[f"twogram_pair_{lang_pair.lower()}"]]
    df_part_twogram_pair.reset_index(drop=True, inplace=True)
    set_part_twogram_pair = set(df_part_twogram_pair[f"twogram_pair_{lang_pair.lower()}"])
else:
    set_part_twogram_pair = set([])  # option for skip Condition 1
# --- Parameter End ---

# while loop code block

twogram_num = word_thresh_num * step_num   # word_thresh_num*step_num minimum: for each word takes two twogram
twogram_pair_num = word_thresh_num * step_num  # word_thresh_num*step_num minimum: for each word takes two twogram pair

while word_end <= word_limit:
    df_word = df_word_all.iloc[word_start:word_end,]
    df_word.reset_index(drop=True, inplace=True)

    # language pair twogram
    word_pair_list = df_pair["dict_entry_main"].to_list()  # *****
    word_list = df_word["word"].to_list()  # ***** 
    ngram_list = []
    for i in word_pair_list:
        for j in word_tokenize(i):
            for k in word_list:
                twogram_1_2 = f"{j} {k}"
                ngram_list.append(twogram_1_2)
                twogram_2_1 = f"{k} {j}"
                ngram_list.append(twogram_2_1)
    df_pair_ngram = pd.DataFrame(ngram_list)
    df_pair_ngram = pd.DataFrame(ngram_list, columns=["twogram"])
    #df_pair_ngram.rename(columns={0:"twogram"}, inplace=True)  # *****
    df_pair_ngram.iloc[:,0] = df_pair_ngram.iloc[:,0].apply(lambda x: remove_repetition(x))
    df_pair_ngram.drop_duplicates(inplace=True)
    df_pair_ngram.reset_index(drop=True, inplace=True)
    df_lang_pair_twogram = pd.merge(df_twogram_sent, df_pair_ngram, how="inner", on="twogram")
    df_lang_pair_twogram.rename(columns={"twogram":f"twogram_pair_{lang_pair.lower()}"}, inplace=True)
    df_lang_pair_twogram.drop_duplicates(inplace=True)
    #df_lang_pair_twogram = df_lang_pair_twogram.head(100)
    set_lang_pair_twogram = set(df_lang_pair_twogram[f"twogram_pair_{lang_pair.lower()}"])
    df_set_result = pd.DataFrame(set_lang_pair_twogram.difference(set_part_twogram_pair))
    df_set_result.rename(columns={0:f"twogram_pair_{lang_pair.lower()}"}, inplace=True)
    df_set_pair_twogram = pd.merge(df_lang_pair_twogram, df_set_result, how="inner", on=f"twogram_pair_{lang_pair.lower()}")

    # twogram
    word_list = df_word["word"].values.tolist()
    data_kind = "twogram"
    twogram_list  = df_twogram_sent.iloc[:,0].values.tolist()
    
    resultlist2 = []

    manager = multiprocessing.Manager()
    resultlist2 = manager.list()
    
    def word_in_wordgroup2(list_var2):
        mergelist = []
        try:
            word = list_var2.split()
        except:
            pass
        var1 = range(len(word))
        for j in var1:
            if word[j] in word_list:
                mergelist.append(word[j])
                if len(mergelist) == len(word):
                        resultlist2.append(list_var2)
                            
    if __name__ == '__main__':
        # with Pool(16) as p:
        with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
            p.map(word_in_wordgroup2, twogram_list) # string_word liste 

    result_list2 = list(resultlist2)
    df_result2 = pd.DataFrame(result_list2)
    df_result2 = pd.DataFrame(result_list2, columns=[f"{data_kind}"])  # *****
    #df_result2 = df_result2.rename(columns = {0: f"{data_kind}"})  # *****
    df_result2.iloc[:,0] = df_result2.iloc[:,0].apply(lambda x: remove_repetition(x)) # **
    df_merge2 = pd.merge(df_result2, df_twogram_sent, how="inner", on=f"{data_kind}")
    df_merge_result2 = df_merge2.sort_values(by="frequency", ascending=False)
    df_merge_result2.drop_duplicates(inplace=True)
    df_merge_result2.reset_index(drop=True, inplace=True)
    df_twogram_result = df_merge_result2
    #df_twogram_result = df_twogram_result.head(100)

    # output
    df_output_result = pd.concat([df_word, df_twogram_result, df_set_pair_twogram], axis=1)

    df_lesson_result = pd.DataFrame(columns=["word","freq_word","twogram","freq_twogram",f"twogram_pair_{lang_pair.lower()}",f"freq_twogram_pair_{lang_pair.lower()}"])
    a = 0
    b = 0

    for i in range(len(df_output_result)):  # *****
        # Insert words and their count 
        try:
            word = df_output_result.iloc[i,0]  # word
            freq_word = df_output_result.iloc[i,1]  # word freq
            df_lesson_result.loc[i,"word"] = word
            df_lesson_result.loc[i,"freq_word"] = freq_word
        except:
            pass
         
        # Insert n grams
        try:
            var1 = df_output_result.iloc[a,2]
            freq_var1 = df_output_result.iloc[a,3]
            if (len(df_lesson_result["twogram"]) < twogram_num): # and (not(word_count(df_lesson_result, word_thresh_num))):
                df_lesson_result.loc[i,"twogram"] = var1
                df_lesson_result.loc[i,"freq_twogram"] = freq_var1
                try:
                    while word_count_bool(df_lesson_result, (word_thresh_num - twogram_thresh_minus), ["twogram"]): # word count result                
                        a += 1
                        var1 = df_output_result.iloc[a,2]
                        freq_var1 = df_output_result.iloc[a,3]
                        df_lesson_result.loc[i,"twogram"] = var1
                        df_lesson_result.loc[i,"freq_twogram"] = freq_var1
                    else:
                        pass
                except:
                    df_lesson_result.loc[i,"twogram"] = np.nan
                    df_lesson_result.loc[i,"freq_twogram"] = np.nan
            else:
                pass
        except:
            pass
        a += 1

        try:
            var2 = df_output_result.iloc[b,4]
            freq_var2 = df_output_result.iloc[b,5]
            if (len(df_lesson_result[f"twogram_pair_{lang_pair.lower()}"]) < twogram_pair_num): # and (not(word_count(df_lesson_result, word_thresh_num))):
                df_lesson_result.loc[i,f"twogram_pair_{lang_pair.lower()}"] = var2
                df_lesson_result.loc[i,f"freq_twogram_pair_{lang_pair.lower()}"] = freq_var2
                try:
                    while word_count_bool(df_lesson_result, (word_thresh_num - twogram_pair_thresh_minus), [f"twogram_pair_{lang_pair.lower()}"]): # word count result                
                        b += 1
                        var2 = df_output_result.iloc[b,4]
                        freq_var2 = df_output_result.iloc[b,5]
                        df_lesson_result.loc[i,f"twogram_pair_{lang_pair.lower()}"] = var2
                        df_lesson_result.loc[i,f"freq_twogram_pair_{lang_pair.lower()}"] = freq_var2
                    else:
                        pass
                except:
                    df_lesson_result.loc[i,f"twogram_pair_{lang_pair.lower()}"] = np.nan
                    df_lesson_result.loc[i,f"freq_twogram_pair_{lang_pair.lower()}"] = np.nan
            else:
                pass
        except:
            pass
        b += 1

    df_lesson_word_count = word_count_result(df_lesson_result, ["twogram",f"twogram_pair_{lang_pair.lower()}"])
    df_lesson_result = pd.merge(df_lesson_result, df_lesson_word_count, how="left", on="word")
    df_lesson_result = df_lesson_result.drop_duplicates()
    df_lesson_result.to_excel(f"{lang_folder}_{lang_pair}_{word_limit}_Word_Step_{step_num}_Result{part_num}.xlsx", index=False)

    word_start += step_num
    word_end += step_num
    part_num += 1 

#### Check Result

In [18]:
df_lesson_result

,word,freq_word,twogram,freq_twogram,twogram_pair_french,freq_twogram_pair_french,word_count
0,bir,18835735.0,ne var,62532.0,evet ekselansları,467,20.0
1,bu,11062659.0,ben de,59972.0,ne sürpriz,401,20.0
2,ne,8025880.0,değil mi,58386.0,pardon ne,382,20.0
3,ve,7766036.0,ben mi,33652.0,çok normal,380,20.0
4,için,5484109.0,ne için,31857.0,ne romantik,370,14.0
...,...,...,...,...,...,...,...
241,NaN,NaN,NaN,NaN,daha basma,4,NaN
242,NaN,NaN,NaN,NaN,drama beni,4,NaN
243,NaN,NaN,NaN,NaN,beni bastın,3,NaN
244,NaN,NaN,NaN,NaN,bazen beni,3,NaN


In [ ]:
#word_count_result(df_lesson_result, ["twogram"])

In [ ]:
#word_count_result(df_lesson_result, [f"twogram_pair_{lang_pair.lower()}"])

#### Read Output File And Multi Write

In [20]:
df_part_11 = pd.read_excel(f"{lang_folder}_{lang_pair}_{word_limit}_Word_Step_7_Result1.xlsx")
df_part_11

,word,freq_word,twogram_pair_french,freq_twogram_pair_french,word_count
0,bir,18835735.0,polis mi,2526,2.0
1,bu,11062659.0,bir numara,1272,2.0
2,ne,8025880.0,bu komik,994,2.0
3,ve,7766036.0,ne duydun,954,1.0
4,için,5484109.0,bebek mi,861,2.0
5,mi,5362714.0,ne planı,835,2.0
6,o,5013838.0,bu normal,828,2.0
7,NaN,NaN,ve motor,776,NaN
8,NaN,NaN,bir şartla,718,NaN
9,NaN,NaN,şans için,305,NaN


In [21]:
df_part_12 = pd.read_excel(f"{lang_folder}_{lang_pair}_{word_limit}_Word_Step_7_Result2.xlsx")
df_part_12

,word,freq_word,twogram_pair_french,freq_twogram_pair_french,word_count
0,ben,4908913.0,çok komik,13097,2.0
1,de,4880315.0,çok basit,4669,1.0
2,çok,4852169.0,evet duydum,3278,2.0
3,ama,4661966.0,evet komutanım,2522,2.0
4,var,4389551.0,ben doktorum,1830,2.0
5,evet,4324786.0,randevum var,916,2.0
6,mı,4001316.0,plan mı,556,2.0
7,NaN,NaN,bomba mı,541,NaN
8,NaN,NaN,ben avukatım,410,NaN
9,NaN,NaN,bomba var,326,NaN


In [22]:
df_part_13 = pd.read_excel(f"{lang_folder}_{lang_pair}_{word_limit}_Word_Step_7_Result3.xlsx")
df_part_13

,word,freq_word,twogram_pair_french,freq_twogram_pair_french,word_count
0,değil,3883885.0,komik değil,1785,2.0
1,da,3610161.0,fazla değil,1756,2.0
2,şey,3602024.0,daha fazla,1738,2.0
3,hayır,3406992.0,hayır komutanım,764,2.0
4,daha,3317577.0,hayır doktor,481,2.0
5,sen,3283654.0,sen defol,176,2.0
6,kadar,2697900.0,sen yat,156,1.0
7,NaN,NaN,komik şey,76,NaN
8,NaN,NaN,daha basit,76,NaN
9,NaN,NaN,şeker şey,70,NaN


In [23]:
df_part_14 = pd.read_excel(f"{lang_folder}_{lang_pair}_{word_limit}_Word_Step_7_Result4.xlsx")
df_part_14

,word,freq_word,twogram_pair_french,freq_twogram_pair_french,word_count
0,bana,2659182.0,beni duydun,3541,1.0
1,yok,2491685.0,seni duydum,1703,2.0
2,onu,2486889.0,onu duydun,1473,2.0
3,seni,2454988.0,bunu duydum,839,2.0
4,beni,2446696.0,problem yok,746,2.0
5,bunu,2445337.0,sinyal yok,742,2.0
6,gibi,2427957.0,seni manyak,279,2.0
7,NaN,NaN,bomba gibi,236,NaN
8,NaN,NaN,bebek gibi,175,NaN
9,NaN,NaN,beni duydu,125,NaN


In [24]:
df_part_21 = pd.read_excel(f"{lang_folder}_{lang_pair}_{word_limit}_Word_Step_28_Result1.xlsx")
df_part_21

,word,freq_word,twogram,freq_twogram,twogram_pair_french,freq_twogram_pair_french,word_count
0,bir,18835735.0,ne var,62532.0,evet ekselansları,467,20.0
1,bu,11062659.0,ben de,59972.0,ne sürpriz,401,20.0
2,ne,8025880.0,değil mi,58386.0,pardon ne,382,20.0
3,ve,7766036.0,ben mi,33652.0,çok normal,380,20.0
4,için,5484109.0,ne için,31857.0,ne romantik,370,14.0
...,...,...,...,...,...,...,...
240,NaN,NaN,NaN,NaN,daha amerikan,4,NaN
241,NaN,NaN,NaN,NaN,daha basma,4,NaN
242,NaN,NaN,NaN,NaN,drama beni,4,NaN
243,NaN,NaN,NaN,NaN,beni bastın,3,NaN


In [ ]:
writer = pd.ExcelWriter(f"{lang_folder}_{lang_pair}_{word_limit}_Word_Result_With_Frequency.xlsx", engine='xlsxwriter')

In [ ]:
df_part_11.to_excel(writer, sheet_name='Word_Part11', index=False)
df_part_12.to_excel(writer, sheet_name='Word_Part12', index=False)
df_part_13.to_excel(writer, sheet_name='Word_Part13', index=False)
df_part_14.to_excel(writer, sheet_name='Word_Part14', index=False)
df_part_21.to_excel(writer, sheet_name='Word_Part21', index=False)

In [ ]:
writer.save()

In [27]:
try:
    writer = pd.ExcelWriter(f"{lang_folder}_{lang_pair}_{word_limit}_Word_Result_With_Frequency.xlsx", engine='xlsxwriter')
    for i in range(1, (condition1_total_part_num+1)):        
        if condition1_dependency:
            df_part_var = pd.read_excel(f"{lang_folder}_{lang_pair}_{word_limit}_Word_Step_{int(condition1_word_limit/condition1_total_part_num)}_Result{i}.xlsx")
            df_part_var.to_excel(writer, sheet_name=f'Word_Part1{i}', index=False)            
        else:
            pass
except:
    pass

df_part_var2 = pd.read_excel(f"{lang_folder}_{lang_pair}_{word_limit}_Word_Step_{step_num}_Result1.xlsx")
df_part_var2.to_excel(writer, sheet_name=f'Word_Part21', index=False)        
writer.save()

#### Output File Word Order

In [28]:
df_part_11

,word,freq_word,twogram_pair_french,freq_twogram_pair_french,word_count
0,bir,18835735.0,polis mi,2526,2.0
1,bu,11062659.0,bir numara,1272,2.0
2,ne,8025880.0,bu komik,994,2.0
3,ve,7766036.0,ne duydun,954,1.0
4,için,5484109.0,bebek mi,861,2.0
5,mi,5362714.0,ne planı,835,2.0
6,o,5013838.0,bu normal,828,2.0
7,NaN,NaN,ve motor,776,NaN
8,NaN,NaN,bir şartla,718,NaN
9,NaN,NaN,şans için,305,NaN


In [29]:
def word_in_twogram(df, list_column, target_column):

    '''word_in_twogram(df, "word", "twogram"):
       df is dataframe, list_column and target_column are 
       dataframe column string name. list_column convert list
       values that are in target column.
    '''
    
    df_select = df[[f"{target_column}"]].dropna()
    df_word_result = pd.DataFrame()
    for i in df[f"{list_column}"].dropna():
        try:
            word_in_twogram = df_select[df_select[f"{target_column}"].str.contains(fr"(?:\s|^){i}(?:\s|$)", na=True)] 
        except:
            pass        
        word_in_twogram.insert(0,"word",i)
        df_word_result = pd.concat([df_word_result,word_in_twogram], axis=0)
    df_word_result.reset_index(drop=True, inplace=True)

    return df_word_result

In [30]:
word_in_twogram(df_part_11, "word", f"twogram_pair_{lang_pair.lower()}") 

,word,twogram_pair_french
0,bir,bir numara
1,bir,bir şartla
2,bu,bu komik
3,bu,bu normal
4,ne,ne duydun
5,ne,ne planı
6,ve,ve motor
7,için,şans için
8,için,bebek için
9,mi,polis mi


In [31]:
df_word_order_11 = word_in_twogram(df_part_11, "word", f"twogram_pair_{lang_pair.lower()}")
df_word_order_12 = word_in_twogram(df_part_12, "word", f"twogram_pair_{lang_pair.lower()}") 
df_word_order_13 = word_in_twogram(df_part_13, "word", f"twogram_pair_{lang_pair.lower()}") 
df_word_order_14 = word_in_twogram(df_part_14, "word", f"twogram_pair_{lang_pair.lower()}")
df_word_order_21 = word_in_twogram(df_part_21, "word", f"twogram") 
df_word_order_212 = word_in_twogram(df_part_21, "word", f"twogram_pair_{lang_pair.lower()}")  

In [32]:
df_word_order_join_11 = df_word_order_11.groupby(["word"])[f"twogram_pair_{lang_pair.lower()}"].apply(", ".join).reset_index()   # df_word_order_11.groupby(["word"])[f"twogram_pair_{lang_pair.lower()}"].transform(lambda x: ','.join(x))
df_word_order_join_12 = df_word_order_12.groupby(["word"])[f"twogram_pair_{lang_pair.lower()}"].apply(", ".join).reset_index()
df_word_order_join_13 = df_word_order_13.groupby(["word"])[f"twogram_pair_{lang_pair.lower()}"].apply(", ".join).reset_index()
df_word_order_join_14 = df_word_order_14.groupby(["word"])[f"twogram_pair_{lang_pair.lower()}"].apply(", ".join).reset_index()
df_word_order_join_211 = df_word_order_21.groupby(["word"])["twogram"].apply(", ".join).reset_index()
df_word_order_join_212 = df_word_order_212.groupby(["word"])[f"twogram_pair_{lang_pair.lower()}"].apply(", ".join).reset_index()
df_word_order_join_21 = pd.merge(df_word_order_join_211, df_word_order_join_212, how="outer", on="word")

In [33]:
df_word_order_join_21

,word,twogram,twogram_pair_french
0,ama,"evet ama, ama ben, ama ne, ama bu, ama sen, ha...","ama general, ama amatörler, ama madam, duydum ..."
1,bana,"bana mı, bana da, bana değil, ve bana, bana bi...","bana spesiyal, bana hamburger, bana vanilyalı,..."
2,ben,"ben de, ben mi, hayır ben, ama ben, ben değil,...","ben patronum, ben bekarım, ben doktor, bazen b..."
3,beni,"beni mi, beni de, beni değil, ve beni, beni ço...","çarp beni, gazla beni, beni ispiyonladı, mesaj..."
4,bir,"bir daha, bu bir, bir şey, bir de, bir mi, ben...","bir mesaj, bir telefon, bir sürpriz, bir fare,..."
5,bu,"bu ne, bu o, bu kadar, ne bu, ama bu, bu da, b...","bu doktor, adres bu, doktor bu, numara bu, bu ..."
6,bunu,"bunu da, ve bunu, bunu için, bunu bana, bunu g...",NaN
7,da,"bu da, bana da, o da, bunu da, onu da, daha da...","motor da, pasaportumu da, doktor da, şanslı da..."
8,daha,"daha değil, bir daha, daha çok, daha var, daha...","daha komik, daha konforlu, daha klas, daha nor..."
9,de,"ben de, sen de, seni de, bir de, evet de, beni...","ceketini de, patates de, polis de, polise de, ..."


In [34]:
writer2 = pd.ExcelWriter(f"{lang_folder}_{lang_pair}_{word_limit}_Word_Join_Result_Without_Frequency.xlsx", engine='xlsxwriter')

In [35]:
df_word_order_join_11.to_excel(writer2, sheet_name='Word_Part11', index=False)
df_word_order_join_12.to_excel(writer2, sheet_name='Word_Part12', index=False)
df_word_order_join_13.to_excel(writer2, sheet_name='Word_Part13', index=False)
df_word_order_join_14.to_excel(writer2, sheet_name='Word_Part14', index=False)
df_word_order_join_21.to_excel(writer2, sheet_name='Word_Part21', index=False)

In [36]:
writer2.save()

In [ ]:
try:
    writer2 = pd.ExcelWriter(f"{lang_folder}_{lang_pair}_{word_limit}_Word_Join_Result_Without_Frequency.xlsx", engine='xlsxwriter')
    for i in range(1, (condition1_total_part_num+1)):        
        if condition1_dependency:
            df_part_var = pd.read_excel(f"{lang_folder}_{lang_pair}_{word_limit}_Word_Step_{int(condition1_word_limit/condition1_total_part_num)}_Result{i}.xlsx")
            df_part_var_order = word_in_twogram(df_part_var, "word", f"twogram_pair_{lang_pair.lower()}")
            df_part_var_order_join = df_part_var_order.groupby(["word"])[f"twogram_pair_{lang_pair.lower()}"].apply(", ".join).reset_index()
            df_part_var_order_join.to_excel(writer2, sheet_name=f'Word_Part1{i}', index=False)          
        else:
            pass
except:
    pass

df_part_var2 = pd.read_excel(f"{lang_folder}_{lang_pair}_{word_limit}_Word_Step_{step_num}_Result1.xlsx")
df_word_order_21 = word_in_twogram(df_part_var2, "word", f"twogram")
df_word_order_212 = word_in_twogram(df_part_var2, "word", f"twogram_pair_{lang_pair.lower()}")
df_word_order_join_211 = df_word_order_21.groupby(["word"])["twogram"].apply(", ".join).reset_index()
df_word_order_join_212 = df_word_order_212.groupby(["word"])[f"twogram_pair_{lang_pair.lower()}"].apply(", ".join).reset_index()
df_word_order_join_21 = pd.merge(df_word_order_join_211, df_word_order_join_212, how="outer", on="word")
df_word_order_join_21.to_excel(writer2, sheet_name='Word_Part21', index=False)        
writer2.save()

#### Copy Move And Delete

In [ ]:
output_file = glob.glob(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_{word_limit}_Word_*.xlsx")
output_file

In [ ]:
for k in output_file:
    source = k # source directory
    destination = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Deployment/Result/3-Adjust Word Level/{lang_folder.capitalize()} {lang_pair.capitalize()}"
    shutil.copy2(source, destination)

In [ ]:
for i in output_file:
    try:
        os.remove(i)
    except:
        pass